In [ ]:
#This notebook is by Anastasia Ruzmaikina for the Kaggle Competition Contradictory, My Dear Watson

If you have two sentences, there are three ways they could be related: one could entail the other, one could contradict the other, or they could be unrelated. Natural Language Inferencing (NLI) is a popular NLP problem that involves determining how pairs of sentences (consisting of a premise and a hypothesis) are related.

Your task is to create an NLI model that assigns labels of 0, 1, or 2 (corresponding to entailment, neutral, and contradiction
) to pairs of premises and hypotheses. To make things more interesting, the train and test set include text in fifteen different languages!

In this notebook, I use Llama-2-7b-hf to analyze how pairs of sentences are related. This notebook has 69.2% accuracy on the validation dataset which the model has not seen before. I read that Llama does not like some foreign languages, and this may be the reason that DebertaV3 outperforms Llama2 on this task.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Install package for inferences
!pip install -qq --no-deps /kaggle/input/daigt-pip/peft-0.6.0-py3-none-any.whl --use-deprecated=legacy-resolver
!pip install -qq --no-deps /kaggle/input/daigt-pip/transformers-4.35.0-py3-none-any.whl --use-deprecated=legacy-resolver
!pip install -qq --no-deps /kaggle/input/daigt-pip/tokenizers-0.14.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl --use-deprecated=legacy-resolver
!pip install -qq --no-deps /kaggle/input/accelerate-and-bitsandbytes/accelerate-0.29.3-py3-none-any.whl --use-deprecated=legacy-resolver
!pip install -qq --no-deps /kaggle/input/accelerate-and-bitsandbytes/bitsandbytes-0.43.1-py3-none-manylinux_2_24_x86_64.whl --use-deprecated=legacy-resolver
!pip install -qq --no-deps /kaggle/input/daigt-pip/optimum-1.14.0-py3-none-any.whl --use-deprecated=legacy-resolver
#!pip install -qq --no-deps /kaggle/input/llm-detect-pip/accelerate-0.24.1-py3-none-any.whl
#!pip install -qq --no-deps /kaggle/input/llm-detect-pip/bitsandbytes-0.41.1-py3-none-any.whl
#!os.register_at_fork()
!pip install -qq --no-dep /kaggle/input/bitsandbytes-accelerate/packages/accelerate-1.0.1-py3-none-any.whl --use-deprecated=legacy-resolver
!pip install -qq --no-dep /kaggle/input/bitsandbytes-accelerate/packages/bitsandbytes-0.42.0-py3-none-any.whl --use-deprecated=legacy-resolver


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

ambiguous option: --no-dep (--no-dependencies, --no-deps?)

Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

ambiguous option: --no-dep (--no-dependencies, --no-deps?)


In [3]:
#!pip install -q -U peft --no-index --find-links ../input/llm-detect-pip/ --use-deprecated=legacy-resolver
#!pip install -q -U accelerate --no-index --find-links ../input/llm-detect-pip/ --use-deprecated=legacy-resolver
#!pip install -q -U bitsandbytes --no-index --find-links ../input/llm-detect-pip/ --use-deprecated=legacy-resolver
#!pip install -q -U transformers --no-index --find-links ../input/llm-detect-pip/ --use-deprecated=legacy-resolver
# Install package for inferences
#!pip install -qq --no-deps /kaggle/input/hf-libraries/bitsandbytes --use-deprecated=legacy-resolver
!pip install -qq --no-deps /kaggle/input/daigt-pip/peft-0.6.0-py3-none-any.whl --use-deprecated=legacy-resolver
#!pip install -qq --no-deps /kaggle/input/daigt-pip/transformers-4.35.0-py3-none-any.whl 
#!pip install -qq --no-deps /kaggle/input/daigt-pip/tokenizers-0.14.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install -qq --no-deps /kaggle/input/daigt-pip/optimum-1.14.0-py3-none-any.whl --use-deprecated=legacy-resolver
!pip install -qq --no-deps /kaggle/input/llm-detect-pip/accelerate-0.24.1-py3-none-any.whl  --use-deprecated=legacy-resolver
!pip install -qq --no-deps /kaggle/input/llm-detect-pip/bitsandbytes-0.41.1-py3-none-any.whl --use-deprecated=legacy-resolver
#!pip install -qq --no-deps /kaggle/input/bitsandbytes-0-42-0 --use-deprecated=legacy-resolver
!pip install -qq --no-deps /kaggle/input/llama3-1-dependencies/dependencies/bitsandbytes-0.43.2-py3-none-manylinux_2_24_x86_64.whl  --use-deprecated=legacy-resolver
!pip install -qq --no-deps /kaggle/input/llama3-1-dependencies/dependencies/accelerate-0.33.0-py3-none-any.whl  --use-deprecated=legacy-resolver

In [4]:
from __future__ import annotations

TARGET_MODEL = '/kaggle/input/llama2-7b-hf' #'/kaggle/input/llama2-7b-hf/Llama2-7b-hf'#"mistralai/Mistral-7B-v0.1"

DEBUG = False

In [5]:
# ====================================================
# Directory settings
# ====================================================
from pathlib import Path

OUTPUT_DIR = Path("./")
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

INPUT_DIR = Path("../input/")

In [6]:
import pandas as pd
train_df = pd.read_csv(INPUT_DIR / "/kaggle/input/contradictory-my-dear-watson/train.csv", sep=',')
test_df = pd.read_csv(INPUT_DIR / "/kaggle/input/contradictory-my-dear-watson/test.csv", sep=',')
external_df = pd.read_csv(INPUT_DIR / "/kaggle/input/contradictory-my-dear-watson/train.csv", sep=',')
train_df['input'] = 'PREMISE: ' + train_df.premise + '; HYPOTHESSIS:' + train_df.hypothesis + ';'
train_df.insert(1, 'input', train_df.pop('input'))
test_df['input'] = 'PREMISE: ' + test_df.premise + '; HYPOTHESSIS:' + test_df.hypothesis + ';'
test_df.insert(1, 'input', test_df.pop('input'))


In [7]:
# rename column generated to label

#train_df = train_df.rename(columns={'label': 'label'})
#test_df = test_df.rename(columns={'label': 'label'})
#external_df = external_df.rename(columns={'label': 'label'})
test_df = test_df.drop(['id','lang_abv', 'language', 'premise', 'hypothesis'], axis=1)
test_df.reset_index(inplace=True, drop=True)
test_df

,input
0,PREMISE: بکس، کیسی، راہیل، یسعیاہ، کیلی، کیلی،...
1,PREMISE: هذا هو ما تم نصحنا به.; HYPOTHESSIS:ع...
2,PREMISE: et cela est en grande partie dû au fa...
3,PREMISE: 与城市及其他公民及社区组织代表就IMA的艺术发展进行对话&amp; HYP...
4,PREMISE: Она все еще была там.; HYPOTHESSIS:Мы...
...,...
5190,PREMISE: نیند نے وعدہ کیا کہ موٹل نے سوال میں ...
5191,PREMISE: The rock has a soft texture and can...
5192,PREMISE: 她目前的存在，并考虑到他与沃佛斯顿争执的本质，那是尴尬的。; HYPOTH...
5193,PREMISE: isn't it i can remember i've only bee...


In [8]:
#train_df = pd.concat([train_df, external_df])
train_df = train_df.drop(['id','lang_abv', 'language', 'premise', 'hypothesis'], axis=1)
train_df.reset_index(inplace=True, drop=True)
print(f"Train dataframe has shape: {train_df.shape}")
train_df.head()

Train dataframe has shape: (12120, 2)


,input,label
0,PREMISE: and these comments were considered in...,0
1,PREMISE: These are issues that we wrestle with...,2
2,PREMISE: Des petites choses comme celles-là fo...,0
3,PREMISE: you know they can't really defend the...,0
4,PREMISE: ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที...,1


In [9]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
X = train_df.loc[:, train_df.columns != "label"]
y = train_df.loc[:, train_df.columns == "label"]

for i, (train_index, valid_index) in enumerate(skf.split(X, y)):
    train_df.loc[valid_index, "fold"] = i
    
print(train_df.groupby("fold")["label"].value_counts())
train_df.head()

fold  label
0.0   0        836
      2        812
      1        776
1.0   0        835
      2        813
      1        776
2.0   0        835
      2        813
      1        776
3.0   0        835
      2        813
      1        776
4.0   0        835
      2        813
      1        776
Name: count, dtype: int64


,input,label,fold
0,PREMISE: and these comments were considered in...,0,1.0
1,PREMISE: These are issues that we wrestle with...,2,1.0
2,PREMISE: Des petites choses comme celles-là fo...,0,1.0
3,PREMISE: you know they can't really defend the...,0,0.0
4,PREMISE: ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที...,1,2.0


In [10]:
# fold0 as valid
valid_df = train_df[train_df["fold"] == 0]
train_df = train_df[train_df["fold"] != 0]
print(train_df.shape)
print(valid_df.shape)

(9696, 3)
(2424, 3)


In [11]:
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType # type: ignore
from transformers import BitsAndBytesConfig
import torch

peft_config = LoraConfig(
    r=4, #
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    target_modules=[
        "q_proj",
        "v_proj"
    ],
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

/opt/conda/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/opt/conda/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [12]:
from transformers import AutoTokenizer, LlamaForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(TARGET_MODEL, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

In [13]:
base_model = LlamaForSequenceClassification.from_pretrained(
    TARGET_MODEL,
    num_labels=3,
    quantization_config=bnb_config,
    device_map={"":0}
)
base_model.config.pretraining_tp = 1 # 1 is 7b
base_model.config.pad_token_id = tokenizer.pad_token_id


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:484: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location=map

In [14]:
model = get_peft_model(base_model, peft_config)

In [15]:
model.print_trainable_parameters()

trainable params: 2,109,440 || all params: 6,609,465,344 || trainable%: 0.03191544081420938


In [16]:
#remove this for actual calculation  here we take a smaller sample of the dataframe to speed up 
train_df = train_df.sample(2000, random_state=42)  #2400
print(train_df.tail())
print(train_df.label.value_counts(), valid_df.label.value_counts())

                                                   input  label  fold
4146   PREMISE: uh-huh uh-huh uh-huh yeah well that's...      1   4.0
11117  PREMISE: He's chosen Meg Ryan.; HYPOTHESSIS:A ...      2   3.0
4287   PREMISE: راودته أفكار دموية حول ذلك وحول أشياء...      1   4.0
11695  PREMISE: แต่แม้ว่าตอนฉันจะเป็นเด็กชาย ฉันอาศัย...      0   3.0
10258  PREMISE: วิทยาลัยพยาบาลต้องการของขวัญเพื่อแสดง...      0   2.0
label
0    715
2    652
1    633
Name: count, dtype: int64 label
0    836
2    812
1    776
Name: count, dtype: int64


In [17]:
valid_df1 = valid_df.sample(frac = 0.4, random_state=42)
valid_df = valid_df.loc[~valid_df.index.isin(valid_df1.index)]
print(train_df.shape)
print(valid_df.shape)
print(valid_df1.shape)

(2000, 3)
(1454, 3)
(970, 3)


In [18]:
# datasets
from datasets import Dataset

# from pandas
train_ds = Dataset.from_pandas(train_df)
valid_ds = Dataset.from_pandas(valid_df)
test_ds = Dataset.from_pandas(test_df)
valid_ds1 = Dataset.from_pandas(valid_df1)


In [19]:
def preprocess_function(examples, max_length=512):
    return tokenizer(examples["input"], truncation=True, max_length=max_length, padding=True)

In [20]:
train_tokenized_ds = train_ds.map(preprocess_function, batched=True)
valid_tokenized_ds = valid_ds.map(preprocess_function, batched=True)
test_tokenized_ds = test_ds.map(preprocess_function, batched=True)
valid_tokenized_ds1 = valid_ds1.map(preprocess_function, batched=True)


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1454 [00:00<?, ? examples/s]

Map:   0%|          | 0/5195 [00:00<?, ? examples/s]

Map:   0%|          | 0/970 [00:00<?, ? examples/s]

In [21]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="longest")

In [22]:
import numpy as np
from sklearn.metrics import accuracy_score, roc_auc_score
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    accuracy_val = accuracy_score(labels, predictions)
    #roc_auc_val = roc_auc_score(labels, predictions)
    
    return {
        "accuracy": accuracy_val,
        #"roc_auc": roc_auc_val,
    }


In [23]:
from transformers import TrainingArguments, Trainer

steps = 5 if DEBUG else 20

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    learning_rate=5e-4,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,
    max_grad_norm=0.3,
    optim='paged_adamw_32bit',
    lr_scheduler_type="cosine",
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    push_to_hub=False,
    warmup_steps=steps,
    eval_steps=steps,
    logging_steps=steps,
    report_to='none' # if DEBUG else 'wandb',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_ds,
    eval_dataset=valid_tokenized_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:451: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
20,1.492000,1.258789,0.358322
40,1.310300,1.365234,0.350069
60,1.182000,1.186523,0.439477
80,1.039000,0.844238,0.592847
100,0.814700,0.786621,0.651994
120,0.822700,0.739746,0.691884


TrainOutput(global_step=125, training_loss=1.0936916275024413, metrics={'train_runtime': 15192.3624, 'train_samples_per_second': 0.132, 'train_steps_per_second': 0.008, 'total_flos': 2.6354104123392e+16, 'train_loss': 1.0936916275024413, 'epoch': 1.0})

In [40]:
#this is to check if the predictions work on a known dataframe not previously seen by the model
preds = trainer.predict(valid_tokenized_ds1)
logits = preds.predictions


In [41]:
# from scipy.special import expit as sigmoid
import numpy as np
def sigmoid(x):
    return 1 / (1 + np.exp(-x))  
logits = sigmoid(logits)
logits = np.argmax(logits, axis=1)
print(logits)
logits.shape, logits[0:5]

[1 1 2 0 0 0 2 1 0 0 2 2 0 0 1 2 2 1 0 1 1 2 0 1 0 0 1 2 1 1 2 0 1 1 1 0 1
 1 2 0 0 1 1 2 0 0 1 0 0 1 0 2 0 0 2 2 0 0 2 0 0 1 1 0 0 1 2 0 0 0 0 2 0 0
 1 0 0 0 0 2 0 1 1 2 1 1 2 0 0 1 1 1 2 1 1 1 1 0 2 0 2 0 0 1 1 2 1 2 0 1 0
 1 0 2 2 1 0 0 2 0 0 1 1 2 0 0 1 1 1 0 0 0 1 1 0 0 0 2 0 0 0 0 0 0 2 2 0 1
 1 1 0 1 1 0 2 0 2 2 2 0 1 2 0 0 0 1 0 0 1 0 1 0 0 2 0 2 0 0 0 0 0 1 2 2 0
 0 2 1 1 0 0 2 0 0 2 0 0 1 0 2 0 2 2 0 1 1 2 0 1 1 0 0 1 0 0 0 1 0 2 1 0 1
 2 1 2 1 2 0 1 1 0 1 0 2 2 0 1 2 1 2 0 1 1 0 2 0 1 0 0 0 2 0 2 2 1 1 2 1 0
 2 1 0 1 1 0 0 1 2 2 2 1 1 2 1 0 1 2 1 0 0 2 1 1 0 0 0 1 0 1 1 0 0 2 2 0 2
 2 0 0 0 1 1 0 0 2 1 0 0 0 0 0 0 2 0 1 1 1 0 0 1 2 2 1 0 0 1 1 1 2 0 1 0 2
 1 2 1 0 0 1 1 2 0 0 2 2 0 0 2 1 1 2 1 0 0 1 0 2 0 0 1 1 0 0 2 1 0 1 0 0 0
 0 0 0 0 1 0 1 0 0 0 2 0 2 2 1 0 2 2 0 1 1 0 1 2 0 0 0 0 0 0 1 1 0 0 0 2 0
 1 1 1 2 1 0 2 2 0 2 0 2 0 2 1 0 2 0 0 1 1 1 0 1 1 1 0 0 0 0 2 2 1 0 0 1 1
 2 1 1 0 0 0 0 0 1 1 0 1 1 0 0 0 0 0 0 0 2 1 2 2 1 1 1 0 0 2 1 0 2 0 0 0 0
 0 2 2 0 2 2 0 1 2 0 2 2 

((970,), array([1, 1, 2, 0, 0]))

In [42]:
#this is to check if the predictions work on a known dataframe not previosly seen by the model
valid_df1['preds'] = logits #np.argmax(probs, axis=1)
valid_df1

,input,label,fold,preds
7459,PREMISE: all right thanks bye bye; HYPOTHESSIS...,2,0.0,1
1862,PREMISE: Từ năm trang xác nhận cá nhân (trái n...,1,0.0,1
7134,"PREMISE: I had rejected it as absurd, neverthe...",2,0.0,2
1946,"PREMISE: Apparently, Greuze wasn't worried abo...",0,0.0,0
2250,PREMISE: right and uh there's usually nobody r...,2,0.0,0
...,...,...,...,...
11379,PREMISE: Arsenic would put poor Emily out of t...,0,0.0,0
653,PREMISE: Cuối cùng anh ta nói: Kẻ giết người--...,2,0.0,1
6147,"PREMISE: To their good fortune, he's proving t...",2,0.0,0
21,PREMISE: Increased saving by current generatio...,0,0.0,0


In [35]:
#this is for the predictions on the test set
preds = trainer.predict(test_tokenized_ds)#.predictions.astype(float)
logits = preds.predictions


In [36]:
logits = preds.predictions
logits

array([[-0.05728, -1.333  , -1.682  ],
       [-1.1875 , -0.537  , -0.02242],
       [ 2.637  , -0.635  , -3.305  ],
       ...,
       [ 0.2399 ,  0.02396, -1.754  ],
       [ 2.98   , -0.297  , -2.715  ],
       [-1.957  , -1.474  ,  1.313  ]], dtype=float16)

In [37]:
# from scipy.special import expit as sigmoid
import numpy as np
def sigmoid(x):
    #if x > -100:
        return 1 / (1 + np.exp(-x))  
   # else:
       # return 0
logits = sigmoid(logits)
logits = np.argmax(logits, axis=1)
logits[:100]

array([0, 2, 0, 1, 2, 2, 1, 0, 1, 1, 0, 1, 0, 2, 0, 1, 0, 0, 0, 2, 2, 2,
       1, 0, 2, 2, 0, 2, 0, 1, 2, 1, 0, 2, 1, 2, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 2, 2, 0, 0, 1, 2, 0, 0, 2, 0, 1, 1, 1, 1, 0, 0, 0, 0,
       2, 2, 0, 0, 1, 1, 0, 1, 0, 2, 0, 2, 0, 1, 2, 2, 1, 0, 0, 1, 1, 2,
       2, 0, 2, 0, 0, 0, 0, 2, 1, 2, 1, 2])

In [38]:

logits.shape, logits[0:5]


((5195,), array([0, 2, 0, 1, 2]))

In [39]:
#this is to provide the sample submission
sub = pd.DataFrame()
test_df = pd.read_csv('/kaggle/input/contradictory-my-dear-watson/test.csv')

sub['id'] = test_df['id']
import math

sub["prediction"] = logits

sub['prediction'] = sub['prediction'].astype('int')
sub.to_csv('/kaggle/working/submission.csv', index=False)
sub[:100]

,id,prediction
0,c6d58c3f69,0
1,cefcc82292,2
2,e98005252c,0
3,58518c10ba,1
4,c32b0d16df,2
...,...,...
95,0540817ced,2
96,740c34dec2,1
97,f25d51d911,2
98,d64a31ab7d,1
